In [1]:
! pip install pandas openpyxl

### lineup 정보 추가와 argument 오타수정

In [2]:
import json
import pandas as pd
import os
import random

import requests
from tqdm import tqdm
from IPython.display import display, Markdown
import time
import math

import itertools
import numpy as np

In [3]:
os.chdir("/home/jovyan/bacillus-shared-sllm")

In [4]:
dir_data = 'datasets/call_plan_classifier'
file_name = '요금제agent_240912.xlsx'
plan_list_name = '요금제목록.xlsx'
lineup_list_name = '요금제라인업목록.xlsx'

In [5]:
tot_data = pd.read_excel(os.path.join(dir_data, file_name)).reset_index(drop=True)

In [6]:
feature_ids = tot_data['Feature ID'].unique()

In [7]:
plan_list_data = pd.read_excel(os.path.join(dir_data, plan_list_name))
plan_list = list(plan_list_data['상품명'])

In [8]:
lineup_list_data = pd.read_excel(os.path.join(dir_data, lineup_list_name))
lineup_list = list(lineup_list_data['요금제라인업'])

# 1. Divide Train Test data

In [9]:
feature_ids

array(['F17-I01', 'F17-I02', 'F17-I11', 'F17-I12', 'F17-I15', 'F17-I13',
       'F17-I14', 'F17-I17', 'F17-I18', 'F17-I19', 'F17-I20', 'F17-I21',
       'F01-I01', 'F01-I02', 'F01-I03', 'F05-I01_01', 'F05_I02',
       'F08-I01-1_E01', 'F08-I01-1_E02', 'F08-I01-1_E03', 'F08-I01-1_E04',
       'F08-I01-1_E05', 'F08-I01-1_E06', 'F08-I01-1_E07', 'F08-I01-1_E08',
       'F08-I01-1_E09', 'F08-I01-1_E10', 'F08-I01-1_E11', 'F08-I01-1_E12',
       'F08-I01-1_E13', 'F08-I01-1_E14', 'F08-I01-1_E15', 'F08-I01-1_E16',
       'F08-I01-1_E17', 'F08-I01-1_E18', 'F08-I01-1_E19', 'F08-I01-1_E20',
       'F08-I01-1_E21', 'F08-I02-1_E01', 'F08-I02-1_E02', 'F08-I02-1_E03',
       'F08-I02-1_E04', 'F08-I02-1_E05', 'F08-I02-1_E06', 'F08-I02-1_E07',
       'F08-I02-1_E08', 'F08-I02-1_E09', 'F08-I02-1_E10', 'F08-I02-1_E11',
       'F08-I02-1_E12', 'F08-I02-1_E13', 'F08-I02-1_E14', 'F08-I02-1_E15',
       'F08-I02-1_E16', 'F08-I02-1_E17', 'F08-I02-1_E18', 'F08-I02-1_E19',
       'F08-I02-1_E20', 'F08-I02-1_E21'

In [10]:
train_num = 7
shot_num = 1

In [11]:
train_index = []
count = 0
for _id in feature_ids:
    _index = list(tot_data.index[tot_data['Feature ID']==_id])
    random.seed(count*100)
    idx = random.sample(_index, train_num)
    train_index += idx
    count += 1

In [12]:
test_index = list(set(tot_data.index) - set(train_index))

In [13]:
train_data = tot_data.iloc[train_index].reset_index(drop=True)
test_data = tot_data.iloc[test_index].reset_index(drop=True)

In [14]:
test_index_list = []
feature_index_list = []
for _id in feature_ids:
    feature_index = list(train_data.index[train_data['Feature ID']==_id])
    combinations = list(itertools.combinations(feature_index, shot_num))
    for comb_idx in range(len(combinations)):
        element = combinations[comb_idx]
        try:
            feature_index_list[comb_idx] += list(element)
            test_index_list[comb_idx] += list(set(feature_index) - set(element))
        except:
            feature_index_list.append(list(element))
            test_index_list.append(list(set(feature_index) - set(element)))
    # feature_index.append(list(train_data.index[train_data['Feature ID']==_id]))
    
test_data_list = []
for i in range(len(test_index_list)):
    test_data_list.append(pd.concat([test_data,train_data.iloc[test_index_list[i]]]).reset_index(drop=True))

# 2. function 정의

In [15]:
FUNCTIONS = [
    {
        "type": "function",
        "function": {
          "name": "my_mobile_plan",
          "description": "Useful when get my mobile plan.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "my_subscribed_add_ons",
          "description": "Useful for providing your subscribed add-ons(가입 된 부가서비스).",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "my_billing_charge",
          "description": "Useful when you need to provide recent billing charges (for last month's usage).",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "my_realtime_billing_charge",
          "description": "Useful when you need to provide real-time billing charges based on current month usage.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "my_realtime_data_usage",
          "description": "Useful when guide real-time data usage/residuals.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "my_shared_data_usage",
          "description": "Useful when you need to provide shared and tethered data usage or residuals.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "my_average_data_usage",
          "description": "Useful for providing average data usage from last 3 months data usage.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "remaining_data_refill_coupons",
          "description": "Useful for guiding about remaining data refill coupons.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "remaining_data_gift",
          "description": "Useful for guiding whether data can be gifted and how many times it remains available.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "t_family_data_usage",
          "description": "Useful for guiding status of 'T가족모아데이터' data usage.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "t_family_data_change_shared_amount",
          "description": "Useful for changing to 'T가족모아데이터' shared data amount.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "remaining_no_contract_plan_points",
          "description": "Useful for checking points remaining on no-contract plans(무약정플랜).",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "billing_charge_analyze_summary",
          "description": "Useful for providing how user's billing history compares to the previous month.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "billing_charge_analyze_discount",
          "description": "Useful for providing details of the discount on billing charges and the increase/decrease compared to last month.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "billing_charge_data_usage",
          "description": "Useful for providing data usage for this month and compare to last month.",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "COMPARE_BETWEEN_MOBILE_PLAN",
          "description": "Useful when compare features of multiple SKTelecom mobile plans.",
          "parameters": {
            "type": "object",
            "properties": {
                "plans": {
                    "description": "List of SKTelecom mobile plans.",
                    "type": "array",
                    "items": {
                      "type": "string"
                    }
                }
              },
              "required": [
                  "plans"
              ]
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "COMPARE_CHANGE_MY_MOBILE_PLAN",
          "description": "Useful when compare when changing your plan.",
          "parameters": {
            "type": "object",
            "properties": {
                "plan": {
                    "description": "SKTelecom mobile plan you want to change.",
                    "type": "string"
                }
              },
              "required": [
                  "plan"
              ]
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "SEARCH_MOBILE_PLAN",
          "description": "This function searches or offer mobile plans based on a specific keyword.",
          "parameters": {
              "type": "object",
              "properties": {
                  "keywords": {
                      "type": "object",
                      "description": "search keywords",
                      "properties": {}
                  }
              }
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "AVAILABLE_MOBILE_PLAN",
          "description": "This function is used to check whether user can subscribe to the given plan.",
          "parameters": {
              "type": "object",
              "properties": {
                  "keywords": {
                      "type": "object",
                      "description": "search keywords",
                      "properties": {}
                  }
              }
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "changable_date_for_plan",
          "description": "Useful for checking how long user have to stay on user's current plan and when user can change it.",
          "parameters": {
              "type": "object",
              "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "plan_auto_change_date",
          "description": "This function is used to inform about when user's plan is automatically changed and what plan it's being changed to.",
          "parameters": {
              "type": "object",
              "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "plan_auto_change_date_alarm",
          "description": "Useful for signing up for an alert when a user can change user's plan.",
          "parameters": {
              "type": "object",
              "properties": {} 
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "recommend_plans",
          "description": "Useful for recommending plans to users.",
          "parameters": {
              "type": "object",
              "properties": {
                  "keywords": {
                      "type": "object",
                      "description": "search keywords",
                      "properties": {}
                  }
              }
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "estimated_billing_charge_compare_to_current",
          "description": "Useful for providing an estimated bill when changing plan or signing up for a subscription.",
          "parameters": {
              "type": "object",
              "properties": {
                  "keywords": {
                      "type": "object",
                      "description": "search keywords",
                      "properties": {}
                  }
              }
            }
        }
    },
    {
        "type": "function",
        "function": {
          "name": "check_plan_availability",
          "description": "This function is used to check whether user can subscribe to the given plan.",
          "parameters": {
              "type": "object",
              "properties": {
                  "keywords": {
                      "type": "object",
                      "description": "search keywords",
                      "properties": {}
                  }
              }
            }
        }
    }

]

In [16]:
functions_desc = ''
function_template = '''
{num}. **{function_name}**:
\t- Description: {function_desc}'''

required_arg_template = '''
\t- **Required Arguments**:'''

optional_arg_template = '''
\t- **Optional Arguments**:'''

no_arg_template = '''
\t- **No arguments**.'''

arg_item_template = '''
\t\t- `{name}` ({type})'''

num = 1
for function in FUNCTIONS:
    desc = function_template.format(num=num, function_name=function['function']['name'], function_desc=function['function']['description'])
    if function['function']['parameters']['properties'] == {}:
        desc += no_arg_template
    else:
        try:
            req_fields = function['function']['parameters']['required']
            desc += required_arg_template
            args = function['function']['parameters']['properties']
            for req_field in req_fields:
                arg = args[req_field]
                if arg['type'] == 'array':
                    item_type = arg["items"]["type"]
                    arg_type = f'array of {item_type}s'
                else:
                    arg_type = arg['items']['type']
                desc += arg_item_template.format(name=req_field, type=arg_type)
            # optional 있는 경우 추가
            diff_set = set(args.keys()) - set(req_fields)
            if len(diff_set) > 0:
                desc += optional_arg_template
                for key in list(diff_set):
                    arg = args[key]
                    if arg['type'] == 'array':
                        item_type = arg["items"]["type"]
                        arg_type = f'array of {item_type}s'
                    else:
                        arg_type = arg['type']
                    desc += arg_item_template.format(name=key, type=arg_type)
        except:
            desc += optional_arg_template
            args = function['function']['parameters']['properties']
            for key in args.keys():
                arg = args[key]
                if arg['type'] == 'array':
                    item_type = arg["items"]["type"]
                    arg_type = f'array of {item_type}s'
                else:
                    arg_type = arg['type']
                desc += arg_item_template.format(name=key, type=arg_type)
    functions_desc += desc
    num += 1

In [17]:
print(functions_desc)


1. **my_mobile_plan**:
	- Description: Useful when get my mobile plan.
	- **No arguments**.
2. **my_subscribed_add_ons**:
	- Description: Useful for providing your subscribed add-ons(가입 된 부가서비스).
	- **No arguments**.
3. **my_billing_charge**:
	- Description: Useful when you need to provide recent billing charges (for last month's usage).
	- **No arguments**.
4. **my_realtime_billing_charge**:
	- Description: Useful when you need to provide real-time billing charges based on current month usage.
	- **No arguments**.
5. **my_realtime_data_usage**:
	- Description: Useful when guide real-time data usage/residuals.
	- **No arguments**.
6. **my_shared_data_usage**:
	- Description: Useful when you need to provide shared and tethered data usage or residuals.
	- **No arguments**.
7. **my_average_data_usage**:
	- Description: Useful for providing average data usage from last 3 months data usage.
	- **No arguments**.
8. **remaining_data_refill_coupons**:
	- Description: Useful for guiding about 

# 3. ARGUMENTS

In [18]:
shared_arguments = {
    "keywords": {
      "type": "object",
      "description": "search keywords",
      "properties": {
        "productName": {
          "default": "null",
          "description": "name of mobile plan",
          "title": "Productname",
          "type": "string"
        },
        "includedVoiceCall": {
          "default": "null",
          "description": "The amount of voice calls provided by the plan",
          "title": "Includedvoicecall",
          "type": "string"
        },
        "includedVideoOrValueAddedCall": {
          "default": "null",
          "description": "The amount of video calls(영상통화) or added calls(부가통화) provided by the plan",
          "title": "Includedvideoorvalueaddedcall",
          "type": "string"
        },
        "includedVoiceCallTospecifiedNumbers": {
          "default": "null",
          "description": "The amount of voice calls to specified numbers (지정통화) provided by the plan",
          "title": "Includedvoicecalltospecifiednumbers",
          "type": "string"
        },
        "includedData": {
          "default": "null",
          "description": "The amount of data. Generally GB units are used.",
          "title": "Includeddata",
          "type": "string"
        },
        "includedDataForSharingAndTethering": {
          "default": "null",
          "description": "The amount of sharing and tethering data.",
          "title": "Includeddataforsharingandtethering",
          "type": "string"
        },
        "optionData": {
          "default": "null",
          "description": "data provided in addition to the basic data provided",
          "title": "Optiondata",
          "type": "string"
        },
        "availableAmountForDataOptionRefill": {
          "default": "null",
          "description": "amount of data refill",
          "title": "Availableamountfordataoptionrefill",
          "type": "string"
        },
        "maximumAmountForSharing": {
          "default": "null",
          "description": "The maximum amount of data that can be shared with others",
          "title": "Maximumamountforsharing",
          "type": "string"
        },
        "includedText": {
          "default": "null",
          "description": "amount of SMS",
          "title": "Includedtext",
          "type": "string"
        },
        "monthlyPrice": {
          "default": "null",
          "description": "amount of monthly price. KRW(원) units are used.",
          "title": "Monthlyprice",
          "type": "string"
        },
        "generation": {
          "default": "null",
          "description": "Available mobile networks (5G, LTE, etc.)",
          "title": "Generation",
          "type": "string"
        },
        "dataQoS": {
          "default": "null",
          "description": "Speed control when you can continue to use data without additional charges after your default data limit has been used up",
          "title": "Dataqos",
          "type": "string"
        },
        "benefit": {
          "default": "null",
          "description": "benefits of the mobile plan",
          "title": "Benefit",
          "type": "string"
        },
        "lineup": {
          "default": "null",
          "description": "The name of the parent group to which the mobile plan belongs",
          "title": "Lineup",
          "type": "string"
        },
        "onboardingChannel": {
          "default": "null",
          "description": "Channel to subscribe to the product",
          "title": "Onboardingchannel",
          "type": "string"
        },
        "onboardingTypeEligibility": {
          "default": "null",
          "description": "Available subscription types",
          "title": "Onboardingtypeeligibility",
          "type": "string"
        },
        "onboardingDevice": {
          "default": "null",
          "description": "device for subscribing to the product",
          "title": "Onboardingdevice",
          "type": "string"
        },
        "onboardingDeviceAlias": {
          "default": "null",
          "description": "alias of device for subscribing to the product",
          "title": "Onboardingdevicealias",
          "type": "string"
        },
        "specialCustomerOnboard": {
          "default": "null",
          "description": "Customer type segment (military, college student, etc.)",
          "title": "Specialcustomeronboard",
          "type": "string"
        }
      }
    },
    "plan": {
        "description": "SKTelecom mobile plan you want to change.",
        "type": "string"
    },
    "plans": {
        "description": "List of SKTelecom mobile plans.",
        "type": "array",
        "items": {
          "type": "string"
        }
    }
}

In [19]:
args_desc = ''
arg_obj_template = '''
{num}. **{arg_name}** (object):
\t- Description: {arg_desc}
\t- Keys:'''

arg_obj_key_default_template = '''
\t\t- {name} ({type}, default: {default}) - {key_desc}'''

arg_obj_key_template = '''
\t\t- {name} ({type}) - {key_desc}'''

arg_simple_template = '''
{num}. **{arg_name}** ({type}):
\t- Description: {arg_desc}'''
num = 1
for arg_name in shared_arguments:
    content = shared_arguments[arg_name]
    if content['type'] == 'object':
        arg = arg_obj_template.format(num=num, arg_name=arg_name, arg_desc=content['description'])
        keys = content['properties']
        for key in keys:
            item = keys[key]
            try:
                default_val = item['default']
                arg += arg_obj_key_default_template.format(name=key, type=item['type'], key_desc=item['description'], default=default_val)
            except:
                arg += arg_obj_key_template.format(name=key, type=item['type'], key_desc=item['description'])
    else:
        if content['type'] == 'array':
            item_type = content["items"]["type"]
            arg_type = f'array of {item_type}s'
        else:
            arg_type = content['type']
        arg = arg_simple_template.format(num=num, arg_name=arg_name, type=arg_type, arg_desc=content['description'])
    args_desc += arg
    num += 1

In [20]:
print(args_desc)


1. **keywords** (object):
	- Description: search keywords
	- Keys:
		- productName (string, default: null) - name of mobile plan
		- includedVoiceCall (string, default: null) - The amount of voice calls provided by the plan
		- includedVideoOrValueAddedCall (string, default: null) - The amount of video calls(영상통화) or added calls(부가통화) provided by the plan
		- includedVoiceCallTospecifiedNumbers (string, default: null) - The amount of voice calls to specified numbers (지정통화) provided by the plan
		- includedData (string, default: null) - The amount of data. Generally GB units are used.
		- includedDataForSharingAndTethering (string, default: null) - The amount of sharing and tethering data.
		- optionData (string, default: null) - data provided in addition to the basic data provided
		- availableAmountForDataOptionRefill (string, default: null) - amount of data refill
		- maximumAmountForSharing (string, default: null) - The maximum amount of data that can be shared with others
		- incl

# 4. System prompt

In [21]:
system_prompt_template = f"""You're a function classifier that needs to identify the appropriate function related to SKTelecom rate plans in order to accurately respond to the user's utterances.
You're actively involved in a three-way conversation with 'user', 'function' and yourself ('assistant').
You must classify the appropriate "function name" and "arguments" according to the user's utterance, and keep the following rules:
1. "Function name" must be classified only from the lists provided below. You SHOULD NEVER GUESS and CREATE something that is not in the defined list.
2. Arguments may or may not be required depending on the selected function.
3. If the selected function has a 'required' field, you must fill in the arguments and send it.
4. Arguments can be inferred from the user's utterance or the previous conversation, and they usually relate to SKTelecom mobile plans or additional services.

### SKTelecom mobile plans
{plan_list}

### SKTelecom mobile plan lineups
{lineup_list}

### Shared Arguments:
These arguments are shared by multiple functions.{args_desc}

### Function Descriptions:{functions_desc}
"""

In [22]:
print(system_prompt_template)

You're a function classifier that needs to identify the appropriate function related to SKTelecom rate plans in order to accurately respond to the user's utterances.
You're actively involved in a three-way conversation with 'user', 'function' and yourself ('assistant').
You must classify the appropriate "function name" and "arguments" according to the user's utterance, and keep the following rules:
1. "Function name" must be classified only from the lists provided below. You SHOULD NEVER GUESS and CREATE something that is not in the defined list.
2. Arguments may or may not be required depending on the selected function.
3. If the selected function has a 'required' field, you must fill in the arguments and send it.
4. Arguments can be inferred from the user's utterance or the previous conversation, and they usually relate to SKTelecom mobile plans or additional services.

### SKTelecom mobile plans
['팅PLUS29요금', '표준요금제', '다이렉트5G 69', '다이렉트5G 62', '다이렉트5G 55', '다이렉트5G 52', '다이렉트5G 48', 

In [23]:
shot_prompt_template = f'''{system_prompt_template}

### EXAMPLES'''

In [24]:
example_template_list = []

In [25]:
example_template_list.append('''
user: "_user_msg_"
assistant: _LLM_output_
''')

In [26]:
# example_template_list.append('''
# {"role": "user", "content": "_user_msg_"},
# {"role": "assistant", "content": _LLM_output_}
# ''')

In [27]:
system_prompt_list_list = []
for ex_idx in range(len(example_template_list)):
    example_template = example_template_list[ex_idx]
    system_prompt_list = []
    for i in range(len(feature_index_list)):
        feature_index = feature_index_list[i]
        examples = ''
        for idx in feature_index:
            element = train_data.iloc[idx]
            temp_example = example_template.replace('_user_msg_',element['Utterance Sentence']).replace('_LLM_output_',element['LLM output'])
            examples += temp_example
            # examples += example_template.format(user_msg = element['Utterance Sentence'], label = element['LLM output'])
        system_prompt_list.append(shot_prompt_template + examples)
    system_prompt_list_list.append(system_prompt_list)

In [28]:
dir_path = 'evaluation/hjs/call_plan_classifier'
for i in range(len(system_prompt_list_list[0])):
    file_name = f'system_prompt_gpt4o_type4_1_240912_{shot_num}shot_comb{i}.txt'
    with open (os.path.join(dir_path, file_name), 'w') as file:
        file.write(system_prompt_list_list[0][i])
# for i in range(len(system_prompt_list_list[1])):
#     file_name = f'system_prompt_gpt4o_type3_2_240912_{shot_num}shot_comb{i}.txt'
#     with open (os.path.join(dir_path, file_name), 'w') as file:
#         file.write(system_prompt_list_list[1][i])

# 5. PET

In [29]:
def gen_PET(
    url="http://172.18.243.211:13100", path="/api/v2/chat/completions/stream", **kwargs
):
    resp = requests.post(
        url + path,
        json={"is_production": False, "transaction_id": "test", **kwargs},
    )
    try:
        result = resp.json()
        result['elapsed'] = resp.elapsed.total_seconds()
    except Exception as e:
        result = resp

    return result

In [30]:
promptid = 124261
path = '/api/v2/chat/completions'

In [31]:
system_prompt = {
        "role": "system",
        "content": ''
}

In [32]:
user_prompt = {
    "role": "user",
    "content": ''
}

In [33]:
chatgpt_list_result_list = []
chatgpt_list_cv_result_list_list = []
for sys_idx in range(len(system_prompt_list_list)):
    system_prompt_list = system_prompt_list_list[sys_idx]
    print('cv check')
    chatgpt_list_cv_result_list = []
    for i in range(len(system_prompt_list)):
        system_prompt['content'] = system_prompt_list[i]
        cv_test_data = test_data_list[i]
        chatgpt_list_result = []
        count = 0
        for idx,element in tqdm(cv_test_data.iterrows()):
            record = {}
            record['input'] = element['Utterance Sentence']
            record['true_response'] = eval(element['LLM output'])
            user_prompt['content'] = element['Utterance Sentence']
            gen_result = gen_PET(
                path=path,
                prompt_id=promptid,
                messages=[system_prompt,user_prompt],
                # tools=FUNCTIONS,
                temperature = 0.
            )
            try:
                record[f'{shot_num}-shot comb{i} prompt'] = eval(gen_result['res']['choices'][0]['message']['content'])
            except:
                record[f'{shot_num}-shot comb{i} prompt'] = gen_result['res']['choices'][0]['message']['content']
            chatgpt_list_result.append(record)
            count += 1
        chatgpt_list_cv_result_list.append(chatgpt_list_result)
    chatgpt_list_cv_result_list_list.append(chatgpt_list_cv_result_list)
    print('='*100)
    
    chatgpt_list_result = []
    count = 0
    for idx,element in tqdm(test_data.iterrows()):
        record = {}
        record['input'] = element['Utterance Sentence']
        record['true_response'] = eval(element['LLM output'])
        user_prompt['content'] = element['Utterance Sentence']
        for i in range(len(system_prompt_list)):
            system_prompt['content'] = system_prompt_list[i]
            gen_result = gen_PET(
                path=path,
                prompt_id=promptid,
                messages=[system_prompt,user_prompt],
                # tools=FUNCTIONS,
                temperature = 0.
            )
            try:
                record[f'{shot_num}-shot comb{i} prompt'] = eval(gen_result['res']['choices'][0]['message']['content'])
            except:
                record[f'{shot_num}-shot comb{i} prompt'] = gen_result['res']['choices'][0]['message']['content']
            # break
        chatgpt_list_result.append(record)
        count += 1
    chatgpt_list_result_list.append(chatgpt_list_result)
    print('='*100)

cv check


541it [09:29,  1.05s/it]
541it [09:05,  1.01s/it]
541it [08:55,  1.01it/s]
541it [09:03,  1.00s/it]
541it [09:14,  1.03s/it]
541it [08:55,  1.01it/s]
359it [05:44,  1.04it/s]


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [34]:
len(system_prompt_list)

7

In [35]:
count

359

In [36]:
len(chatgpt_list_result)

359

In [39]:
chatgpt_list_result = chatgpt_list_result[:357]

In [40]:
len(chatgpt_list_result)

357

In [41]:
pre_count = 357

In [43]:
    count = 0
    for idx,element in tqdm(cv_test_data.iterrows()):
        if pre_count > count:
            count += 1
            continue
        record = {}
        record['input'] = element['Utterance Sentence']
        record['true_response'] = eval(element['LLM output'])
        user_prompt['content'] = element['Utterance Sentence']
        gen_result = gen_PET(
            path=path,
            prompt_id=promptid,
            messages=[system_prompt,user_prompt],
            # tools=FUNCTIONS,
            temperature = 0.
        )
        try:
            record[f'{shot_num}-shot comb{i} prompt'] = eval(gen_result['res']['choices'][0]['message']['content'])
        except:
            record[f'{shot_num}-shot comb{i} prompt'] = gen_result['res']['choices'][0]['message']['content']
        chatgpt_list_result.append(record)
        count += 1
    chatgpt_list_cv_result_list.append(chatgpt_list_result)

541it [03:40,  2.45it/s] 


In [44]:
chatgpt_list_cv_result_list_list.append(chatgpt_list_cv_result_list)

In [45]:
    chatgpt_list_result = []
    count = 0
    for idx,element in tqdm(test_data.iterrows()):
        record = {}
        record['input'] = element['Utterance Sentence']
        record['true_response'] = eval(element['LLM output'])
        user_prompt['content'] = element['Utterance Sentence']
        for i in range(len(system_prompt_list)):
            system_prompt['content'] = system_prompt_list[i]
            gen_result = gen_PET(
                path=path,
                prompt_id=promptid,
                messages=[system_prompt,user_prompt],
                # tools=FUNCTIONS,
                temperature = 0.
            )
            try:
                record[f'{shot_num}-shot comb{i} prompt'] = eval(gen_result['res']['choices'][0]['message']['content'])
            except:
                record[f'{shot_num}-shot comb{i} prompt'] = gen_result['res']['choices'][0]['message']['content']
            # break
        chatgpt_list_result.append(record)
        count += 1
    chatgpt_list_result_list.append(chatgpt_list_result)

187it [22:08,  7.10s/it]


# 6. Make prompt result

In [46]:
def generate_metric_df_batch(chatgpt_list_result):
    df = pd.DataFrame(chatgpt_list_result)

    peft_metric_intent_list = []
    peft_metric_argument_list = []

    cols = list(df.columns)[2:]
    for j in range(len(cols)):
        peft_metric_intent_list.append([])
        peft_metric_argument_list.append([])

    for i in range(df.shape[0]):
        # print('#' * 100)
        # print(i)
        # print("input: ", df["input"][i])

        try:
            # true_res = json.loads(df["true_response"][i])["content"]
            true_res = df["true_response"][i]
            # true_res_agent = true_res["agent"]
            true_res_agent = true_res["name"]
            try:
                true_res_intent = eval(true_res["arguments"])
            except:
                true_res_intent = true_res["arguments"]

        except:
            true_res = None
            true_res_agent = None
            true_res_intent = None

        # 정답값이 존재할 경우
        for j in range(len(cols)):
            if true_res != None:
                try:
                    # peft_res = df[f'{shot_num[j]}-shot prompt'][i]
                    if type(df[cols[j]][i]) == dict:
                        peft_res = df[cols[j]][i]
                    else:
                        peft_res = eval(df[cols[j]][i])
                    peft_res_agent = peft_res["name"]
                    try:
                        peft_res_intent = eval(peft_res["arguments"])
                    except:
                        peft_res_intent = peft_res["arguments"]
                except:
                    peft_res = None
                    peft_res_agent = None
                    peft_res_intent = None
                # print("peft_res: ", peft_res)

                peft_metric_intent_list[j].append(str(true_res_agent) == str(peft_res_agent))
                peft_metric_argument_list[j].append(true_res_intent == peft_res_intent)

            # 정답값이 존재하지 않을 경우
            else:
                peft_metric_intent_list[j].append(False)
                peft_metric_argument_list[j].append(False)


    for j in range(len(cols)):
        df[f'{cols[j]} intent metric'] = list(map(int, peft_metric_intent_list[j]))
        df[f'{cols[j]} argument metric'] = list(map(int, peft_metric_argument_list[j]))

    return df

In [47]:
for rslt_idx in range(len(chatgpt_list_result_list)):
    df_result = generate_metric_df_batch(chatgpt_list_result_list[rslt_idx])
    df_result.to_excel(f'evaluation/hjs/call_plan_classifier/output_chatGPT4o_240912_v4_example_form{rslt_idx+1}_{shot_num}shot_optim_test.xlsx', index=False)

In [48]:
for rslt_idx in range(len(chatgpt_list_cv_result_list_list)):
    chatgpt_list_cv_result_list = chatgpt_list_cv_result_list_list[rslt_idx]
    for i in range(len(chatgpt_list_cv_result_list)):
        df_result = generate_metric_df_batch(chatgpt_list_cv_result_list[i])
        df_result.to_excel(f'evaluation/hjs/call_plan_classifier/output_chatGPT4o_240912_v4_example_form{rslt_idx+1}_{shot_num}shot_comb{i}_optim_cvtest.xlsx', index=False)